In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI迁移：AutoML表格二元分类

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-automl-tabular-binary-classification-online-prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-automl-tabular-binary-classification-online-prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"> 在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/migration/sdk-automl-tabular-binary-classification-online-prediction.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"> 在Vertex AI Workbench中打开
    </a>
  </td>                                                                                               
</table>

## 概述

本教程演示了如何使用Vertex AI SDK创建表格二元分类模型，并使用AutoML模型进行在线预测。

了解更多关于[迁移到Vertex AI](https://cloud.google.com/vertex-ai/docs/start/migrating-to-vertex-ai)。 了解更多关于[表格数据的分类](https://cloud.google.com/vertex-ai/docs/tabular-data/classification-regression/overview)。

### 目标

在本教程中，您将使用Vertex AI SDK从Python脚本创建一个AutoML表格二分类模型，并部署到在线预测。您也可以使用`gcloud`命令行工具或通过Cloud Console在线创建和部署模型。

本教程使用以下谷歌云机器学习服务和资源：

- Vertex AI 管理的数据集
- Vertex AI 训练
- Vertex AI 终端
- Vertex AI 预测

执行的步骤包括：

- 创建一个Vertex `Dataset`资源。
- 训练模型。
- 查看模型评估。
- 将`Model`资源部署到一个服务`Endpoint`资源。
- 进行预测。
- 取消部署`Model`。

数据集

本教程使用的数据集是[银行营销](https://pantheon.corp.google.com/storage/browser/_details/cloud-ml-tables-data/bank-marketing.csv)。该数据集不需要进行任何特征工程。您在本教程中使用的数据集版本存储在公共云存储桶中。

成本

本教程使用Google Cloud的计费组件：

* Vertex AI
* 云存储

了解[Vertex AI的定价](https://cloud.google.com/vertex-ai/pricing)和[云存储的定价](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您预计的使用情况生成成本估算。

## 安装

安装以下所需的包以执行此笔记本。

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform fsspec gcsfs

### 仅限协作：取消以下单元格的注释以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

在你开始之前

### 设置您的项目ID

**如果您不知道您的项目ID**，请尝试以下方法:
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面: [查找项目ID](https://support.google.com/googleapi/answer/7014113)

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

#### 区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

UUID

如果您正在进行现场教程会话，您可能正在使用一个共享的测试账户或项目。为了避免用户在创建的资源之间的名称冲突，您为每个实例会话创建一个 UUID，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的Google云帐户

根据您的Jupyter环境，您可能需要手动进行验证。请按照以下相关说明进行操作。

**1. Vertex AI Workbench**
* 不需要进行任何操作，因为您已经验证过了。

**2. 本地JupyterLab实例，请取消注释并运行：**

In [ ]:
# ! gcloud auth login

3. 合作，取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

请看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples为您的服务账户授予Cloud Storage权限。

创建一个云存储桶

创建一个存储桶来存储中间产物，比如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有在您的存储桶尚不存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

### 导入库并定义常量

In [ ]:
import google.cloud.aiplatform as aip
import pandas as pd

## 初始化 Vertex AI SDK for Python

为您的项目和相应的存储桶初始化 Vertex AI SDK for Python。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

#### 云存储培训数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://cloud-ml-tables-data/bank-marketing.csv"

快速查看您的数据

本教程使用存储在公共云存储桶中的 Bank Marketing 数据集的一个版本，使用 CSV 索引文件。

首先快速查看数据。您可以通过计算 CSV 索引文件中的行数（`wc -l`）来统计示例的数量，然后查看前几行。

您还需要在训练过程中知道标签列的标题名称，该名称保存为 `label_column`。对于该数据集，标签列是 CSV 文件中的最后一列。

In [ ]:
count = ! gsutil cat $IMPORT_FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $IMPORT_FILE | head

heading = ! gsutil cat $IMPORT_FILE | head -n1
label_column = str(heading).split(",")[-1].split("'")[0]
print("Label Column Name", label_column)
if label_column is None:
    raise Exception("label column missing")

创建数据集

### [datasets.create-dataset-api](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api)

### [数据集创建API](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api)

创建数据集

接下来，使用 `TabularDataset` 类的 `create` 方法创建 `Dataset` 资源，需要传入以下参数：

- `display_name`： `Dataset` 资源的可读名称。
- `gcs_source`：将一个或多个数据集索引文件导入到 `Dataset` 资源中。
- `bq_source`：或者，从BigQuery表中导入数据项到 `Dataset` 资源中。

此操作可能需要几分钟时间。

In [ ]:
dataset = aip.TabularDataset.create(
    display_name="Bank Marketing" + "_" + UUID, gcs_source=[IMPORT_FILE]
)

print(dataset.resource_name)

INFO：google.cloud.aiplatform.datasets.dataset：创建表格数据集
INFO：google.cloud.aiplatform.datasets.dataset：创建表格数据集支持的LRO：projects/759209241365/locations/us-central1/datasets/5048649531491614720/operations/752335608625496064
INFO：google.cloud.aiplatform.datasets.dataset：已创建TabularDataset。资源名称：projects/759209241365/locations/us-central1/datasets/5048649531491614720
INFO：google.cloud.aiplatform.datasets.dataset：要在另一个会话中使用此TabularDataset：
INFO：google.cloud.aiplatform.datasets.dataset：ds = aiplatform.TabularDataset('projects/759209241365/locations/us-central1/datasets/5048649531491614720')
projects/759209241365/locations/us-central1/datasets/5048649531491614720

训练一个模型

### [training.automl-api] (https://cloud.google.com/vertex-ai/docs/training/automl-api)

### [训练. 自动机器学习 API] (https://cloud.google.com/vertex-ai/docs/training/automl-api)

### 创建和运行训练管道

要训练一个AutoML模型，您需要执行两个步骤：1）创建一个训练管道，2）运行该管道。

#### 创建训练管道

可以使用`AutoMLTabularTrainingJob`类创建一个AutoML训练管道，使用以下参数：

- `display_name`：`TrainingJob`资源的可读名称。
- `optimization_prediction_type`：要为模型训练的任务类型。
  - `classification`：一个表格分类模型。
  - `regression`：一个表格回归模型。
- `column_transformations`：（可选）要应用于输入列的转换。
- `optimization_objective`：要最小化或最大化的优化目标。
  - 二元分类：
    - `minimize-log-loss`
    - `maximize-au-roc`
    - `maximize-au-prc`
    - `maximize-precision-at-recall`
    - `maximize-recall-at-precision`
  - 多类别分类：
    - `minimize-log-loss`
  - 回归：
    - `minimize-rmse`
    - `minimize-mae`
    - `minimize-rmsle`

实例化的对象是训练管道的有向无环图（DAG）。

In [ ]:
job = aip.AutoMLTabularTrainingJob(
    display_name="bank_" + UUID,
    optimization_prediction_type="classification",
    optimization_objective="minimize-log-loss",
)

print(job)

*示例输出：*

<google.cloud.aiplatform.training_jobs.AutoMLTabularTrainingJob对象，位于0x7fc3b6c90f10处>

#### 运行训练管道

接下来，您可以通过调用方法 `run` 并传入以下参数来运行 DAG 以开始训练作业：

- `dataset`: 用于训练模型的 `Dataset` 资源。
- `model_display_name`: 训练模型的可读名称。
- `training_fraction_split`: 用于训练的数据集百分比。
- `test_fraction_split`: 用于测试（留置数据）的数据集百分比。
- `validation_fraction_split`: 用于验证的数据集百分比。
- `target_column`: 要训练的列的名称。
- `budget_milli_node_hours`: （可选）最大训练时间，以毫小时为单位（1000 = 小时）。
- `disable_early_stopping`: 如果设置为 `True`，服务可能会在整个预算使用之前完成训练，如果服务认为无法进一步改善模型目标测量。

`run` 方法执行完成后会返回 `Model` 资源。

训练管道的执行将需要最多 20 分钟。

In [ ]:
model = job.run(
    dataset=dataset,
    model_display_name="bank_" + UUID,
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=1000,
    disable_early_stopping=False,
    target_column=label_column,
)

示例输出：

    INFO: google.cloud.aiplatform.training_jobs:未提供列变换，因此现在从数据集中检索列以设置默认列变换。
    INFO: google.cloud.aiplatform.training_jobs:已设置以下列的类型“auto”列变换：['Day'，'Month'，'MaritalStatus'，'Job'，'Balance'，'Housing'，'Contact'，'Duration'，'Education'，'Previous'，'Loan'，'PDays'，'POutcome'，'Age'，'Default'，'Campaign']。
    INFO: google.cloud.aiplatform.training_jobs:查看培训：
    https://console.cloud.google.com/ai/platform/locations/us-central1/training/4626265542611697664？project=759209241365
    INFO: google.cloud.aiplatform.training_jobs：AutoMLTabularTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/4626265542611697664 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    INFO: google.cloud.aiplatform.training_jobs：AutoMLTabularTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/4626265542611697664 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    INFO: google.cloud.aiplatform.training_jobs：AutoMLTabularTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/4626265542611697664 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    INFO: google.cloud.aiplatform.training_jobs：AutoMLTabularTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/4626265542611697664 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    INFO: google.cloud.aiplatform.training_jobs：AutoMLTabularTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/4626265542611697664 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    ...
    INFO: google.cloud.aiplatform.training_jobs：AutoMLTabularTrainingJob 运行完成。资源名称：projects/759209241365/locations/us-central1/trainingPipelines/3915928254510268416
    INFO: google.cloud.aiplatform.training_jobs：项目/759209241365/locations/us-central1/models/6324223757200130048 处提供的模型。

## 评估模型

### [projects.locations.models.evaluations.list](https://cloud.devsite.corp.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.models.evaluations/list)

### [项目位置模型评估列表](https://cloud.devsite.corp.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.models.evaluations/list)

## 评估模型评分
在您的模型训练完成后，您可以查看其评估分数。

首先，您需要获取对新模型的引用。与数据集一样，您可以使用在部署模型时创建的模型变量的引用，或者列出项目中的所有模型。

In [ ]:
# Get model resource ID
models = aip.Model.list(filter="display_name=bank_" + UUID)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

*示例输出：*

    名称："projects/759209241365/locations/us-central1/models/623915674158235648/evaluations/4280507618583117824"
    指标模式URI："gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml"
    指标 {
      结构值 {
        字段 {
          键："auPrc"
          值 {
            数值:0.9891107
          }
        }
        字段 {
          键："confidenceMetrics"
          值 {
            列表值 {
              值 {
                结构值 {
                  字段 {
                    键："precision"
                    值 {
                      数值:0.2
                    }
                  }
                  字段 {
                    键："recall"
                    值 {
                      数值:1.0
                    }
                  }
                }
              }

进行批量预测

### [predictions.batch-prediction](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

### [批处理预测](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

制作测试项目

您可以使用合成数据作为测试数据项目。不要担心我们使用的是合成数据 - 我们只是想演示如何进行预测。

制作批处理输入文件

现在制作一个批处理输入文件，将其存储在您的本地云存储桶中。与图像、视频和文本不同，表格的批处理输入文件仅支持CSV格式。对于CSV文件，您需要：

- 第一行是带有特征（字段）标题名称的标题。
- 每个剩余行是具有相应特征值的单独预测请求。

例如：

    "feature_1", "feature_2". ...
    value_1, value_2, ...

In [ ]:
! gsutil cat $IMPORT_FILE | head -n 1 > tmp.csv
! gsutil cat $IMPORT_FILE | tail -n 10 >> tmp.csv

! cut -d, -f1-16 tmp.csv > batch.csv

gcs_input_uri = BUCKET_URI + "/test.csv"

! gsutil cp batch.csv $gcs_input_uri

### 进行批量预测请求

现在您的模型资源已经训练好了，您可以通过调用 batch_predict() 方法来进行批量预测，参数如下：

- `job_display_name`: 批量预测作业的人类可读名称。
- `gcs_source`: 一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`: 用于存储批量预测结果的云存储位置。
- `instances_format`: 输入实例的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `predictions_format`: 输出预测的格式，可以是'csv'或'jsonl'。默认为'jsonl'。
- `sync`: 如果设置为True，则调用会阻塞，直到异步批量作业完成。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="bank_" + UUID,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    instances_format="csv",
    predictions_format="csv",
    sync=False,
)

print(batch_predict_job)

示例输出：

INFO：google.cloud.aiplatform.jobs：正在创建BatchPredictionJob
<google.cloud.aiplatform.jobs.BatchPredictionJob object at 0x7f806a6112d0> 正在等待上游依赖项完成。
INFO：google.cloud.aiplatform.jobs：已创建BatchPredictionJob。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296
INFO：google.cloud.aiplatform.jobs：要在另一个会话中使用此BatchPredictionJob：
INFO：google.cloud.aiplatform.jobs：bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296')
INFO：google.cloud.aiplatform.jobs：查看批量预测作业：
https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/5110965452507447296?project=759209241365
INFO：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296 当前状态：
JobState.JOB_STATE_RUNNING

### 等待批量预测作业完成

接下来，等待批量作业完成。或者，可以在`batch_predict()`方法中将参数`sync`设置为`True`，以阻塞直到批次预测作业完成。

In [ ]:
batch_predict_job.wait()

示例输出：

    信息：google.cloud.aiplatform.jobs：已创建BatchPredictionJob。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328
    信息：google.cloud.aiplatform.jobs：要在另一个会话中使用此BatchPredictionJob：
    信息：google.cloud.aiplatform.jobs：bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328')
    信息：google.cloud.aiplatform.jobs：查看批量预测作业：
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/181835033978339328?project=759209241365
    信息：google.cloud.aiplatform.jobs：批量预测作业projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：批量预测作业projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328当前状态：
    JobState.JOB_STATE_RUNNING
    ...
    信息：google.cloud.aiplatform.jobs：批量预测作业projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328当前状态：
    JobState.JOB_STATE_SUCCEEDED
    信息：google.cloud.aiplatform.jobs：批量预测作业运行已完成。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328

获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果将写入您在批量预测请求中指定的云存储输出桶中。您可以调用iter_outputs()方法来获取生成结果的每个云存储文件的列表。每个文件包含一个或多个以CSV格式表示的预测请求：

- CSV标题+预测标签
- CSV行+预测，每个预测请求一个

In [ ]:
bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction.results"):
        prediction_results.append(blob.name)

for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    df = pd.read_csv(gfile_name)
    print(f"File name: {gfile_name}")
    print("Prediction: \n\n\n\n")
    print(df)
    print("\n\n\n")

文件名：gs://vertex-ai-devaip-5j22pmou/prediction-bank_5j22pmou-2022_08_24T01_05_46_028Z/prediction.results-00005-of-00008.csv
預測：

年龄        工作      婚姻状态    教育     默认值     余额      住房     贷款
57     蓝领     已婚     中等教育    否        668     否       否

    联系方式    天数     月份     持续时间     宣传活动    上次联系间隔    之前的联系次数    上一个结果
电话    17      十一月     508         4           -1             0              未知

    存款1分数    存款2分数
0.847498   0.152502

进行在线预测

### [predictions.deploy-model-api](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)

### [模型部署API](https://cloud.google.com/vertex-ai/docs/predictions/deploy-model-api)

## 部署模型

接下来，将您的模型部署到在线预测中。 要部署模型，您需要调用`deploy`方法，并传入以下参数：

- `machine_type`：计算机的类型。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

示例输出：

    INFO: google.cloud.aiplatform.models: 创建端点
    INFO: google.cloud.aiplatform.models: 创建端点支持的 LRO: projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/4087251132693348352
    INFO: google.cloud.aiplatform.models: 端点已创建。资源名称：projects/759209241365/locations/us-central1/endpoints/4867177336350441472
    INFO: google.cloud.aiplatform.models: 要在另一个会话中使用此端点：
    INFO: google.cloud.aiplatform.models: endpoint = aiplatform.Endpoint('projects/759209241365/locations/us-central1/endpoints/4867177336350441472')
    INFO: google.cloud.aiplatform.models: 将模型部署到端点：projects/759209241365/locations/us-central1/endpoints/4867177336350441472
    INFO: google.cloud.aiplatform.models: 部署端点模型支持的 LRO: projects/759209241365/locations/us-central1/endpoints/4867177336350441472/operations/1691336130932244480
    INFO: google.cloud.aiplatform.models: 端点模型已部署。资源名称：projects/759209241365/locations/us-central1/endpoints/4867177336350441472

[预测.online-预测-autoML](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-automl)

制作测试项目

您将合成数据用作测试数据项。不用担心我们使用合成数据 - 我们只是想演示如何进行预测。

In [ ]:
INSTANCE = {
    "Age": "58",
    "Job": "managment",
    "MaritalStatus": "married",
    "Education": "teritary",
    "Default": "no",
    "Balance": "2143",
    "Housing": "yes",
    "Loan": "no",
    "Contact": "unknown",
    "Day": "5",
    "Month": "may",
    "Duration": "261",
    "Campaign": "1",
    "PDays": "-1",
    "Previous": "0",
    "POutcome": "unknown",
}

### 进行预测

现在您的`Model`资源已部署到一个`Endpoint`资源，您可以通过向`Endpoint`资源发送预测请求来进行在线预测。

#### 请求

每个实例的格式为：

    [feature_list]

由于`predict()`方法可以接受多个项目（实例），请将您的单个测试项目发送为一个测试项目列表。

#### 响应

从`predict()`调用得到的响应是一个包含以下条目的Python字典：

- `ids`: 每个预测请求的内部分配的唯一标识符。
- `displayNames`: 每个类标签的类名。
- `confidences`: 每个类标签的预测概率，介于0和1之间。
- `deployed_model_id`: 执行预测的部署过的`Model`资源的Vertex AI标识符。

In [ ]:
instances_list = [INSTANCE]

prediction = endpoint.predict(instances_list)
print(prediction)

*示例输出：*

    预测(predictions=[{'classes': ['1', '2'], 'scores': [0.9966018199920654, 0.003398159053176641]}], 部署模型ID='6741053013336522752', 解释=None)

## 撤销模型

当您完成预测时，您可以从`Endpoint`资源中撤销模型。这将取消所有计算资源，并停止部署模型的计费。

In [ ]:
endpoint.undeploy_all()

# 清理

要清理此项目中使用的所有Google Cloud资源，可以删除用于本教程的[Google Cloud项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除本教程中创建的各个资源：

- 数据集
- 模型
- 终端
- AutoML训练作业
- 批处理作业
- Cloud Storage存储桶

In [ ]:
import os

# Delete the dataset using the Vertex dataset object
dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the endpoint using the Vertex endpoint object
endpoint.delete()

# Delete the AutoML or Pipeline trainig job
job.delete()

# Delete the batch prediction job using the Vertex batch prediction object
batch_predict_job.delete()

delete_bucket = False
if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI